In [7]:

import numpy as np
import pandas as pd

In [8]:
train_path="/home/u131168/mh_one_api/data/train.csv"
ftrain_path="/home/u131168/mh_one_api/data/train_split/full_traind.csv"

test_path="/home/u131168/mh_one_api/data/test.csv"
submission_path="/home/u131168/mh_one_api/data/submission.csv"

train_data=pd.read_csv(train_path)
ftrain_data=pd.read_csv(ftrain_path)

test_data=pd.read_csv(test_path)
submission_data=pd.read_csv(submission_path)


In [9]:
# story=ftrain_data.iloc[0,0]
# question=ftrain_data.iloc[0,1]
# ftrain_data.iloc[:15]

# ftrain_data.iloc[:,2].to_csv("/home/u131168/mh_one_api/data/train_split/train_output.csv",)


In [10]:

!pip install bitsandbytes
!pip install accelerate
!pip install scipy
!pip install transformers
!pip install peft
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer
import torch
# model_path="/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_finetuned_model/checkpoint-36000"
# model_path="/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_ss/checkpoint-7000"
model_path="/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_ft_v1/checkpoint-32900"


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
custom_model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_path, )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_ft_v1/checkpoint-32600'. Use `repo_type` argument if needed.

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_path="google/flan-t5-xl"
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# ft5_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

KeyboardInterrupt: 

In [ ]:
# import torch
# prompt = "Today I believe we can finally"
# prompt = [["Please answer to the following question."," Who is going to be the next john wick d'or?"], ]
# # print(question,story)
# prompt=[[story,question]]
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# # sample up to 30 tokens
# torch.manual_seed(0)  # doctest: +IGNORE_RESULT
# outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=30)

# pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True,clean_up_tokenization_spaces=True)
# # ['Today I believe we can finally get rid of discrimination," said Rep. Mark Pocan (D-Wis.).\n\n"Just look at the']

In [ ]:
from pprint import pprint

# p_count=50
psi,pei=50,55
prompts = ftrain_data.loc[psi:pei,['input','instruction']].values.tolist()
t_prompts=[]
for p in prompts:
    context=str(p[0]).replace(r"\n",'.')
    question=p[1]
    t_prompts+=[f"paragraph: {context}\n\n Answer the following question from the paragraph, if you cant find the answer say unknown : Question: {question}"]
    # t_prompts+=[f"input: {context}\n\ninstruction: {question}"]

prompts=t_prompts
# prompts = ftrain_data.loc[:p_count,['input','instruction']].values.tolist()
# pprint(prompts)

In [ ]:
res=[]

In [ ]:
input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=20)
pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True)

KeyboardInterrupt: 

In [ ]:
# import math
# input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# # input_ids=[input_ids]
# # sample up to 30 tokens
# torch.manual_seed(0)  # doctest: +IGNORE_RESULT
# outputs = ft5_model.generate(input_ids=input_ids, do_sample=True, max_length=20,)
# # print(sum(list(sum(outputs.scores))))
# pred_ft5=tokenizer.batch_decode(outputs, skip_special_tokens=True,)
# # pprint(prompts[5])
# pred_ft5

['Grief',
 'i.',
 'December 17',
 'To ask something to his grandmother',
 'he was composed',
 'it was fun',
 'Madison Square Garden',
 'Lightfoot',
 'leave',
 'i.',
 'CSKA Moscow',
 'Germanic',
 'Grand Cayman, Cayman Brac and Little Cayman',
 'Shahadah',
 'What the time was?',
 'Founded in 1565 by the Portuguese,',
 '3',
 '(ii)',
 'irremediable',
 'Zahra Clare Baker',
 'no',
 'South Waziristan province',
 'the Secretary of State',
 'Simón Bolvar',
 'Josiah Graham,',
 'Finland',
 'Knights of the Bath',
 'Tom to look past the fountain',
 'Whoa!',
 'Vancouver is consistently named as one of the top five worldwide cities for livability and quality of',
 'none are known to be related',
 'A).',
 'The searchlight',
 'He is devastated and horrified by the death of the teen',
 'yes',
 'I.',
 'He was completely exhausted, completely dehydrated, suffered significant weight loss, somewhere up to 50%.',
 'I broke one leg and broke the ankle in my other leg',
 '[B].',
 "John's mother.",
 'It took he

In [ ]:
pred_custom_df=pd.DataFrame(pd.Series(pred_custom,name="pred_custom"))
# pred_ft5_df=pd.DataFrame(pd.Series(pred_ft5,name="pred_ft5"))
temp_df=pd.concat([ftrain_data.iloc[psi:pei].reset_index(),train_data.iloc[psi:pei,4].reset_index(),pred_custom_df,],axis=1)
temp_df
# pprint(temp_df.iloc[1,0])

NameError: name 'pred_custom' is not defined

In [ ]:
# pred_custom_df=pd.DataFrame(pd.Series(pred_custom,name="pred_custom"))
# pred_ft5_df=pd.DataFrame(pd.Series(pred_ft5,name="pred_ft5"))
pred_ft5_df=pd.read_csv("/home/u131168/mh_one_api/data/sub_ft5.csv")
temp_df=pd.concat([test_data.iloc[:p_count],pred_ft5_df[:p_count]],axis=1)
temp_df

,Story,Question,Answer
0,"CHAPTER I \n\nON THE HOUSEBOAT \n\n""Say, Tom, ...",What was Sam's mood?,Anxious
1,The Uralic languages (; sometimes called Urali...,By how many?,125 million
2,The University of Wisconsin–Madison (also know...,How many schools and colleges does it have?,20
3,(CNN) -- The unexpected resignation of David P...,What happened?,David Petraeus remained stoically aloof as a s...
4,"CHAPTER THIRTY-TWO \n\nTENDER TROUBLES \n\n""Jo...",Why does Mrs. March not force her children's c...,She wants to wait
5,The New York Yankees are an American professio...,When did the team start playing in Yankee Stad...,1923
6,South Ossetia () is a partially recognised sta...,Who does Russia not allow to enter there?,European Union
7,(CNN) -- The megayacht that Steve Jobs commiss...,How much money did Steve Jobs commission for t...,100-million-euro
8,"CHAPTER XV \n\nNow that Gordon was gone, at an...",Does he think the reason she gave are conflict...,yes
9,A high-profile murder case involving one of Am...,For the murder of whom?,__


In [ ]:
test_data

,Story,Question
0,"CHAPTER I \n\nON THE HOUSEBOAT \n\n""Say, Tom, ...",What was Sam's mood?
1,The Uralic languages (; sometimes called Urali...,By how many?
2,The University of Wisconsin–Madison (also know...,How many schools and colleges does it have?
3,(CNN) -- The unexpected resignation of David P...,What happened?
4,"CHAPTER THIRTY-TWO \n\nTENDER TROUBLES \n\n""Jo...",Why does Mrs. March not force her children's c...
...,...,...
28543,"CHAPTER III. \n\n""Nice customs curt'sy to grea...",Who was the Archbishop of Toledo that was of t...
28544,CHAPTER XI. \n\nIn the little dining-room of t...,Who lived there?
28545,"Chapter XXX. \n\n""I shall go on through all et...",is he fair?
28546,Chapter XXXIV \n\nViolence \n\n\n\nIt had been...,Where was he to stay until this trip?


In [ ]:
temList=[0 for i in range(28548-len(res))]
submission_data=pd.Series(res+temList,name="Answer")
submission_data=pd.DataFrame(submission_data)
submission_data.to_csv("/home/u131168/mh_one_api/data/submissionflant5base.csv",index=False)
submission_data

,Answer
0,Anxious
1,125 million
2,20
3,David Petraeus remained stoically aloof as a s...
4,She wants to wait
...,...
28543,0
28544,0
28545,0
28546,0
